In [78]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from skimage.transform import resize
import matplotlib.pyplot as plt
from skimage.data import shepp_logan_phantom
import cv2

matrix = pd.read_csv('matrix_40d.csv', header=None).values
result = pd.read_csv('result_40d.csv', header=None).values
N, M = matrix.shape
print(N, M)
print(result.shape)
x = np.zeros(N)
print(x.shape)
phantom = shepp_logan_phantom()
phantom_resized = resize(phantom, (128, 128), mode='reflect', anti_aliasing=True)
phantom_vector = phantom_resized.flatten()
lr = 0.9
iterations = 30
directions = 40
img_n = 128
scaler = MinMaxScaler(feature_range=(0, 255))
img = np.zeros((iterations * directions, img_n, img_n), np.uint8)
img_3 = np.zeros((iterations * directions, img_n, img_n, 3), np.uint8)
matrix = matrix.transpose()


16384 5120
(5120, 1)
(16384,)


/Users/fangxk2003/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/skimage/color/colorconv.py:984: RuntimeWarning: divide by zero encountered in matmul
  return rgb @ coeffs
/Users/fangxk2003/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/skimage/color/colorconv.py:984: RuntimeWarning: overflow encountered in matmul
  return rgb @ coeffs
/Users/fangxk2003/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/skimage/color/colorconv.py:984: RuntimeWarning: invalid value encountered in matmul
  return rgb @ coeffs


In [112]:
x = np.zeros(N)

In [135]:
x_copy = x.copy()

In [109]:
x = x_copy.copy()

In [133]:
iterations = 100
lr = 1.5
for i in range(iterations):
    for d in range(directions):
        for j in range(img_n):
            ai = matrix[d * img_n + j]
            vi = np.dot(ai, x)
            correction = (result[d * img_n + j] - vi) / (np.linalg.norm(ai) ** 2 + 1e-8)
            x += lr * ai * correction
    mse = np.mean(np.square(x - phantom_vector))
    print(f"Iteration {i + 1}, MSE: {mse}")

Iteration 1, MSE: 0.005370302963349881
Iteration 2, MSE: 0.005370301606482783
Iteration 3, MSE: 0.00537030047525309
Iteration 4, MSE: 0.00537029939282427
Iteration 5, MSE: 0.0053702983570550426
Iteration 6, MSE: 0.005370297363997189
Iteration 7, MSE: 0.00537029641434093
Iteration 8, MSE: 0.00537029550255644
Iteration 9, MSE: 0.00537029463058979
Iteration 10, MSE: 0.005370293794008042
Iteration 11, MSE: 0.005370292991025749
Iteration 12, MSE: 0.005370292219214789
Iteration 13, MSE: 0.0053702914752918555
Iteration 14, MSE: 0.005370290758359573
Iteration 15, MSE: 0.005370290065528808
Iteration 16, MSE: 0.005370289396097419
Iteration 17, MSE: 0.005370288747753433
Iteration 18, MSE: 0.0053702881193999925
Iteration 19, MSE: 0.005370287509406418
Iteration 20, MSE: 0.005370286916238528
Iteration 21, MSE: 0.005370286338802625
Iteration 22, MSE: 0.005370285775427427
Iteration 23, MSE: 0.005370285225188335
Iteration 24, MSE: 0.005370284686494765
Iteration 25, MSE: 0.0053702841582090825
Iteration 

In [136]:
iterations = 100
lr = 2
W = np.sum(matrix, axis = 0)
V = np.sum(matrix, axis = 1)
for i in range(iterations):
    xx = np.zeros_like(x)
    for d in range(directions):
        for j in range(img_n):
            ai = matrix[d * img_n + j]
            vi = np.dot(ai, x)
            correction = (result[d * img_n + j] - vi) / (V[d * img_n + j] + 1e-8)
            xx += lr * (ai * correction) / (W + 1e-8)
    x += xx
    # for d in range(directions):
    #     mat_clip = matrix[:, d * img_n : (d + 1) * img_n]
    #     add = np.zeros((N, img_n))
    #     result_clip = result[d * img_n : (d + 1) * img_n]
    #     delta = result_clip
    #     for rays in range(img_n):
    #         delta[rays] -= np.dot(mat_clip[:, rays], x)
    #     # normalize mat_clip by each column
    #     sum_matrix = np.sum(mat_clip, axis=0)
    #     sum_matrix[sum_matrix == 0] = 1
    #     for rays in range(img_n):
    #         # normalize each column of mat_clip
    #         add[:, rays] = mat_clip[:, rays] / sum_matrix[rays] * delta[rays]
    #     # normalize mat_clip by each line
    #     sum_matrix = np.sum(mat_clip, axis=1)
    #     sum_matrix[sum_matrix == 0] = 1
    #     for pixel in range(N):
    #         # normalize each line of mat_clip
    #         add[pixel, :] = add[pixel, :] / sum_matrix[pixel]
    #     x += lr * np.sum(add, axis=1)
    mse = np.mean(np.square(x - phantom_vector))
    print("MSE:", mse)
print(lr)

MSE: 0.005370193337041025
MSE: 0.0053701931809208014
MSE: 0.005370193027100919
MSE: 0.0053701928755346805
MSE: 0.005370192726176648
MSE: 0.005370192578982599
MSE: 0.005370192433909492
MSE: 0.00537019229091543
MSE: 0.005370192149959619
MSE: 0.005370192011002337
MSE: 0.0053701918740048996
MSE: 0.005370191738929627
MSE: 0.005370191605739813
MSE: 0.005370191474399695
MSE: 0.005370191344874425
MSE: 0.005370191217130043
MSE: 0.005370191091133446
MSE: 0.0053701909668523685
MSE: 0.005370190844255348
MSE: 0.005370190723311713
MSE: 0.005370190603991548
MSE: 0.005370190486265676
MSE: 0.005370190370105636
MSE: 0.005370190255483664
MSE: 0.00537019014237267
MSE: 0.0053701900307462185
MSE: 0.00537018992057851
MSE: 0.00537018981184436
MSE: 0.005370189704519188
MSE: 0.005370189598578993
MSE: 0.005370189494000338
MSE: 0.00537018939076034
MSE: 0.005370189288836645
MSE: 0.00537018918820742
MSE: 0.005370189088851335
MSE: 0.005370188990747552
MSE: 0.005370188893875706
MSE: 0.005370188798215896
MSE: 0.005370

In [35]:
for i in range(iterations):
    for d in range(directions):
        # change img[i * directions + d] to 3 channels
        img_3[i * directions + d] = cv2.merge([img[i * directions + d]] * 3)
        cv2.putText(img_3[i * directions + d],  str(i) + "," + str(d), (5, 10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1)
out = cv2.VideoWriter('output_video.avi', cv2.VideoWriter_fourcc(*'MJPG'), 30, (img_n, img_n))
print(img_3.shape)
for frame in img_3:
    out.write(frame)
out.release()

(1200, 128, 128, 3)


In [36]:
cv2.imwrite('last_frame_40d.png', img_3[-1])

True